In [1]:
import ee, os
import geemap
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
Map = geemap.Map()
sand = ee.Image("projects/soilgrids-isric/sand_mean").divide(10)
clay = ee.Image("projects/soilgrids-isric/clay_mean").divide(10)
silt = ee.Image("projects/soilgrids-isric/silt_mean").divide(10)
ph = ee.Image("projects/soilgrids-isric/phh2o_mean").divide(10)
soil = [clay, silt, sand, ph]
soil_name = ['clay', 'silt', 'sand', 'ph']


Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
ph.bandNames().getInfo()

['phh2o_0-5cm_mean',
 'phh2o_5-15cm_mean',
 'phh2o_15-30cm_mean',
 'phh2o_30-60cm_mean',
 'phh2o_60-100cm_mean',
 'phh2o_100-200cm_mean']

In [4]:
excel = pd.read_excel(r"SOC change after LUCC(数据提取).xlsx")
data = excel[['ID', 'Lat', 'Lon', 'Upper depth (cm)', 'Lower depth (cm)']]
data

,ID,Lat,Lon,Upper depth (cm),Lower depth (cm)
0,1,25.512000,78.534000,0.0,15.0
1,2,25.512000,78.534000,15.0,30.0
2,3,25.512000,78.534000,30.0,45.0
3,4,43.416667,82.833333,0.0,10.0
4,5,43.416667,82.833333,10.0,20.0
...,...,...,...,...,...
1374,1375,-12.500000,33.550000,10.0,20.0
1375,1376,-12.500000,33.550000,0.0,10.0
1376,1377,-12.500000,33.550000,10.0,20.0
1377,1378,-12.500000,33.550000,0.0,10.0


In [7]:
# 定义计算未知深度的土壤值及更新深度范围的函数
def calculation(image, down, up, round):
    # 将影像各个波段的值以深度为键，加入进字典中，以便后续用深度调用影像
    bands = image.bandNames().getInfo()   
    dic = {0: image.select(bands[0]), 5: image.select(bands[1]), 15: image.select(bands[2]),
            30: image.select(bands[3]), 60: image.select(bands[4]), 100: image.select(bands[5]),}

    # 初始化pos为集合，清除重复值
    pos = set()
    # 为了将 down 和 up 前后的深度加入集合，用于计算未知深度土壤值，暴力添加选中深度的前后深度
    for index, i in enumerate(round):
        if down < i < up:
            pos.add(i)
            if index > 0:    # 如果 index 大于0，说明前面有值，添加进集合
                pos.add(round[index - 1])
            if index < len(round) - 1:    # 如果 index 小于标准土壤深度数减1，说明后面有值，添加进集合
                pos.add(round[index + 1])
    pos = sorted(pos)    # 将集合转为列表后按大小排序

    # 利用两点式计算出未知土壤深度的值
    down_value = (dic[pos[1]].subtract(dic[pos[0]])).multiply(((down - pos[0]) / (pos[1] - pos[0]))).add(dic[pos[0]])
    up_value = (dic[pos[-1]].subtract(dic[pos[-2]])).multiply(((up - pos[-2]) / (pos[-1] - pos[-2]))).add(dic[pos[-2]])

    # 将计算出的值加入进字典中，以便后续用深度调用影像
    dic[down] = down_value
    dic[up] = up_value
    
    # 更新列表，将超出范围的头和尾去除，把 down 和 up 加入列表
    pos = sorted([down] + [x for x in pos[1:-1]] + [up])
    print(pos)
    # 如此一来，dic 字典中包含了影像所有波段和计算出的 down 和 up 的影像，以深度为键，而 pos 则包含所有需要计算的深度
    return dic, pos

# 定义计算最终影像的函数
def func(down, up):
    round = [0, 5, 15, 30, 60, 100]    # 初始化标准土壤深度
    lst = []    # 初始化列表，用于存储不同土壤属性的最终结果，以便最终转换成 Image
    
    # 遍历所需计算的影像
    for index, image in enumerate(tqdm(soil, desc='计算土壤属性')):

        dic, pos = calculation(image, down, up, round)

        # 初始化值为0的影像，用于承接各个梯形的面积
        total_value = ee.Image(0)
        for i in range(len(pos) - 1):
            x1, x2 = pos[i], pos[i + 1]    # 获取底，为土壤深度
            h1, h2 = dic[x1], dic[x2]    # 获取高，为字典中土壤深度对应的影像
            area = h1.add(h2).multiply(x2 - x1).multiply(0.5)    # 计算面积
            total_value = total_value.add(area)

        # 计算单个土壤属性的最终结果，重命名后加入列表中
        image_value = total_value.multiply(1 / (up - down))
        lst.append(image_value.rename(soil_name[index]))

    # 将计算出的所有土壤属性转成一张影像，波段为所需的各个土壤属性
    final_image = ee.Image(lst)
    return final_image

In [32]:
a = func(0, 50)

计算土壤属性:  25%|██▌       | 1/4 [00:01<00:04,  1.35s/it]

[0, 5, 15, 30, 50]


计算土壤属性:  50%|█████     | 2/4 [00:01<00:01,  1.27it/s]

[0, 5, 15, 30, 50]


计算土壤属性:  75%|███████▌  | 3/4 [00:03<00:01,  1.06s/it]

[0, 5, 15, 30, 50]


计算土壤属性: 100%|██████████| 4/4 [00:04<00:00,  1.12s/it]

[0, 5, 15, 30, 50]


In [30]:
data.iloc[16:17,:]

,ID,Lat,Lon,Upper depth (cm),Lower depth (cm)
16,17,-10.479231,-40.08569,0.0,50.0


In [31]:
points = geemap.df_to_ee(data.iloc[16:17,:], latitude='Lat', longitude='Lon')

In [33]:
test = geemap.extract_values_to_points(points, a, scale=30, crs='epsg:4326', tileScale=16)

In [34]:
test.getInfo()['features'][0]['properties']

{'ID': 17,
 'Lat': -10.4792307257339,
 'Lon': -40.0856897626794,
 'Lower depth (cm)': 50,
 'Upper depth (cm)': 0,
 'clay': 32.593333333333334,
 'ph': 6.22,
 'sand': 48.90666666666666,
 'silt': 18.555}

In [50]:
# 土壤深度加权
def calculation(image, down, up):
    # 将影像各个波段的值以标准层为键，加入进字典中，以便后续调用影像
    bands = image.bandNames().getInfo()   
    dic = {(0,5): image.select(bands[0]), (5,15): image.select(bands[1]), (15,30): image.select(bands[2]),
            (30,60): image.select(bands[3]), (60,100): image.select(bands[4]), (100,200): image.select(bands[5]),}
    total_value = ee.Image(0)
    indexs = []
    for i in dic.keys():
        if down < i[1] and up > i[0]:
            indexs.append(i)
    
    # 各种情况
    if len(indexs) == 1:
        total_value = dic[indexs[0]]
    else:
        for i, index in enumerate(indexs):
            if i == 0:    # 首个
                value = dic[index].multiply((index[1]-down) / (up-down))
            elif i == len(indexs)-1:    # 最后一个
                value = dic[index].multiply((up-index[0]) / (up-down))
            else:    # 中间部分
                value = dic[index].multiply((index[1]-index[0]) / (up-down))

            total_value = total_value.add(value)

    return total_value

def func(down, up):
    lst = []    # 初始化列表，用于存储不同土壤属性的最终结果，以便最终转换成 Image
    for index, image in enumerate(tqdm(soil, desc='计算土壤属性')):
        cal_image = calculation(image, down, up)
        lst.append(cal_image.rename(soil_name[index]))
    
    final_image = ee.Image(lst)
    return final_image

In [51]:
a = func(0, 50)
points = geemap.df_to_ee(data.iloc[16:17,:], latitude='Lat', longitude='Lon')
test = geemap.extract_values_to_points(points, a, scale=30, crs='epsg:4326', tileScale=16)

计算土壤属性: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


In [52]:
test.getInfo()['features'][0]['properties']

{'ID': 17,
 'Lat': -10.4792307257339,
 'Lon': -40.0856897626794,
 'Lower depth (cm)': 50,
 'Upper depth (cm)': 0,
 'clay': 32.18,
 'ph': 6.26,
 'sand': 49.290000000000006,
 'silt': 18.57}